In [1]:
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Point, LineString, MultiLineString
from tqdm.notebook import tqdm
import numpy as np

In [2]:
df_dupl = pd.read_csv('intervs_st.csv',sep=';',encoding='windows-1251')

In [3]:
df = df_dupl.drop_duplicates(subset=df_dupl.columns).reset_index(drop=True)

In [4]:
df = df.sort_values(by=['населенный_пункт',
                        'тип_тс','номер_маршрута']).reset_index(drop=True)

In [5]:
lst_freq = []
i=0
for i in range(len(df)):
    freq = round((60 / df['интерв_ср_буд'][i]),1)
    lst_freq.append(freq)
# 

In [6]:
df['частота_ТС_в_час'] = lst_freq

In [7]:
gdf_lines1 = gpd.read_file('./rts_ya/gdf_line1_СТН.json',encoding='utf-8')

In [8]:
gdf_lines2 = gpd.read_file('./rts_ya/gdf_line_sam_tol_new.json',encoding='utf-8')

In [9]:
gdf_lines1[gdf_lines1.line_id.isin(gdf_lines2.line_id)]

,line_id,direction,geometry
156,2193179417,0,"LINESTRING (50.18772 53.21483, 50.18809 53.214..."
747,2227623077,0,"LINESTRING (49.48976 53.48649, 49.49112 53.486..."
748,2227623077,1,"LINESTRING (49.84604 53.48551, 49.84600 53.485..."


In [10]:
gdf_lines_all = gdf_lines1.append(gdf_lines2[~gdf_lines2.line_id.isin(gdf_lines1.line_id)])
gdf_lines_all = gdf_lines_all.drop_duplicates(subset=['line_id','direction'])
gdf_lines_all = gdf_lines_all.reset_index(drop=True)

In [11]:
gdf_lines_frw = gdf_lines_all[gdf_lines_all.direction == 0].reset_index(drop=True)

In [12]:
df = df.rename(columns={'link_id_yandex_frw':'line_id'})

In [19]:
df['line_id'] = df['line_id'].astype(str)

In [13]:
len(df)

313

In [28]:
full_geo_interv = df.merge(gdf_lines_frw[['line_id','geometry']],how='left',on=['line_id'])

In [34]:
full_geo_interv = gpd.GeoDataFrame(full_geo_interv)
full_geo_interv.crs='epsg:4326'

In [36]:
full_geo_interv.to_file('full_geo_interv.json',driver='GeoJSON',encoding='utf-8')

In [39]:
rail_full_geo_interv = full_geo_interv[
    full_geo_interv.тип_тс == 'трамвай'].reset_index(drop=True)

In [40]:
not_rail_full_geo_interv = full_geo_interv[
    full_geo_interv.тип_тс != 'трамвай'].reset_index(drop=True)

In [42]:
rail_full_geo_interv.to_file('rail_full_geo_interv.json',driver='GeoJSON',encoding='utf-8')

In [43]:
not_rail_full_geo_interv.to_file('not_rail_full_geo_interv.json',driver='GeoJSON',encoding='utf-8')

In [52]:
df_stops1 = pd.read_csv('./rts_ya/df_stop_seq1_СТН.csv',sep=';',encoding='utf-8')

In [47]:
df_stops2 = pd.read_csv('./rts_ya/df_stop_seq_sam_tol_new.csv',sep=';',encoding='utf-8')

In [58]:
df_stop_all = df_stops1.append(df_stops2)
df_stop_all = df_stop_all[df_stop_all.line_id.isin(full_geo_interv.line_id)]
df_stop_all = df_stop_all.drop_duplicates(subset=['line_id','direction','stop_sequence'])
df_stop_all = df_stop_all.reset_index(drop=True)
df_stop_all['line_id'] = df_stop_all['line_id'].astype(str)

In [180]:
full_geo_interv.head(3)

,line_id,населенный_пункт,тип_тс,тип_маршрута,номер_маршрута,наим_маршрута,интерв_ср_буд,частота_ТС_в_час,geometry
0,2193178991,Самара,автобус,муниципальный,1,"а/ст ""Красная Глинка"" - Ж/д вокзал (Поликлиника)",14.0,4.3,"LINESTRING (50.16919 53.38468, 50.16979 53.384..."
1,2193178998,Самара,автобус,пригородный,101а,"г. Самара (а/ст ""Аврора"") - с. Лопатино",45.0,1.3,"LINESTRING (50.19138 53.19107, 50.19141 53.190..."
2,2193179000,Самара,автобус,пригородный,103к,"г. Самара (Площадь им. Кирова) - Совхоз ""Юбиле...",80.0,0.8,"LINESTRING (50.26398 53.21837, 50.26431 53.218..."


In [181]:
np_fgi = full_geo_interv.to_numpy()
ind_geo = list(full_geo_interv.columns).index('geometry')
ind_tpts = list(full_geo_interv.columns).index('тип_тс')
ind_freq = list(full_geo_interv.columns).index('частота_ТС_в_час')
ind_lnid = list(full_geo_interv.columns).index('line_id')
ind_rtnmbr = list(full_geo_interv.columns).index('номер_маршрута')

In [182]:
bg_lst_segm = []
bg_lst_tpts = []
bg_lst_freq = []
bg_lst_wkt = []

bg_lst_rtnmbr = []
bg_lst_lineid = []
i=0
for i in tqdm(range(len(np_fgi))):
# for i in range(2):
    one_line = np_fgi[i,ind_geo]
    one_tpts = np_fgi[i,ind_tpts]
    one_freq = np_fgi[i,ind_freq]
    one_lineid = np_fgi[i,ind_lnid]
    one_rtnmbr = np_fgi[i,ind_rtnmbr]
    
    intrsect = one_line.intersection(one_line)
    lst_segm_new = list(intrsect)
    lst_one_tpts = []
    lst_one_freq = []
    lst_one_wkt = []
    
    lst_one_lineid = []
    lst_one_rtnmbr = []
    for j in range(len(lst_segm_new)):
        if ((one_tpts == 'троллейбус') | (one_tpts == 'автобус')):
            tp_ts_grp = 'not_rail'
        else:
            tp_ts_grp = 'rail'
        #
        lst_one_tpts.append(tp_ts_grp)
        lst_one_freq.append(one_freq)
        lst_one_wkt.append(lst_segm_new[j].to_wkt())
        lst_one_lineid.append(one_lineid)
        lst_one_rtnmbr.append(one_rtnmbr)
    #
    bg_lst_segm = bg_lst_segm + lst_segm_new
    bg_lst_tpts = bg_lst_tpts + lst_one_tpts
    bg_lst_freq = bg_lst_freq + lst_one_freq
    bg_lst_wkt = bg_lst_wkt + lst_one_wkt
    bg_lst_rtnmbr = bg_lst_rtnmbr + lst_one_rtnmbr
    bg_lst_lineid = bg_lst_lineid + lst_one_lineid
# 

In [183]:
df_seg = gpd.GeoDataFrame(data=list(zip(bg_lst_tpts, bg_lst_freq, bg_lst_wkt,
                                        bg_lst_rtnmbr,bg_lst_lineid)), 
                          geometry=bg_lst_segm, 
                          columns=['type_ts','frequency','geo_wkt','route_number','line_id'])
df_seg.crs='epsg:4326'

In [184]:
df_seg.head(3)

,type_ts,frequency,geo_wkt,route_number,line_id,geometry
0,not_rail,4.3,LINESTRING (50.1691939999999974 53.38468400000...,1,2193178991,"LINESTRING (50.16919 53.38468, 50.16979 53.38482)"
1,not_rail,4.3,LINESTRING (50.1697860000000020 53.38482100000...,1,2193178991,"LINESTRING (50.16979 53.38482, 50.16946 53.38539)"
2,not_rail,4.3,LINESTRING (50.1694630000000004 53.38538700000...,1,2193178991,"LINESTRING (50.16946 53.38539, 50.16886 53.38526)"


In [185]:
grp_seg = pd.DataFrame(df_seg.groupby(['geo_wkt','type_ts']).sum()['frequency']).reset_index()
print(len(grp_seg))

54227


In [186]:
grp_seg = grp_seg.sort_values(by=['type_ts','frequency'],ascending=False)

In [187]:
grp_seg = grp_seg.merge(df_seg[['geo_wkt','geometry']].drop_duplicates(subset=['geo_wkt']), 
                        how='left', on=['geo_wkt']).reset_index(drop=True)
print(len(grp_seg))

54227


In [188]:
grp_seg = gpd.GeoDataFrame(grp_seg)
grp_seg.crs='epsg:4326'

In [ ]:
# get all routes in segment

In [212]:
np_gs = grp_seg.to_numpy()
ind_wkt = list(grp_seg.columns).index('geo_wkt')
ind_tpts = list(grp_seg.columns).index('type_ts')

np_ds = df_seg.to_numpy()
ind_wkt2 = list(df_seg.columns).index('geo_wkt')
ind_tpts2 = list(df_seg.columns).index('type_ts')
ind_rtnb2 = list(df_seg.columns).index('route_number')
ind_lnd2 = list(df_seg.columns).index('line_id')

lst_rts_in_seg = []
lst_lineid_in_seg = []
i=0
for i in tqdm(range(len(np_gs))):
# for i in range(10):
    one_wkt = np_gs[i,ind_wkt]
    one_tpts = np_gs[i,ind_tpts]
    prt_df = np_ds[np.where((np_ds[:,ind_wkt2] == one_wkt) * (np_ds[:,ind_tpts2] == one_tpts))]
    
    lst_lineid = list(set(list(prt_df[:,ind_lnd2])))
    lst_rtnmbr = list(set(list(prt_df[:,ind_rtnb2])))
    lst_rts_in_seg.append(str(lst_rtnmbr).replace('[','').replace(']',''))
    lst_lineid_in_seg.append(str(lst_lineid).replace('[','').replace(']',''))
#
print(len(lst_rts_in_seg))
print(len(lst_lineid_in_seg))
# 3452/54227 [01:40<20:16, 41.73it/s]


54227
54227


In [216]:
grp_seg['rtnmbr_in_seg'] = lst_lineid_in_seg
grp_seg['lineid_in_seg'] = lst_rts_in_seg

In [217]:
grp_seg.head(3)

,geo_wkt,type_ts,frequency,geometry,rtnmbr_in_seg,lineid_in_seg
0,LINESTRING (50.2311729999999983 53.22144099999...,rail,32.6,"LINESTRING (50.23117 53.22144, 50.22938 53.22062)","'11', '19', '21', '13', '7'","'2193179383', '2193179005', '2193179041', '219..."
1,LINESTRING (50.2347520000000003 53.22307599999...,rail,32.6,"LINESTRING (50.23475 53.22308, 50.23296 53.22226)","'11', '19', '21', '13', '7'","'2193179383', '2193179005', '2193179041', '219..."
2,LINESTRING (50.2222059999999999 53.21684199999...,rail,26.3,"LINESTRING (50.22221 53.21684, 50.22255 53.21657)","'11', '19', '23', '2', '21', '7'","'2193179383', '2193179005', '2193179041', '219..."


In [218]:
grp_seg.to_file('./interv/grp_seg_ST.json',driver='GeoJSON',encoding='utf-8')